In [2]:
import cv2
import numpy as np
np.set_printoptions(threshold=np.inf)
from matplotlib import pyplot as plt
from os.path import isfile, join
from os import listdir
from sklearn import preprocessing
from skimage import filters

# TASK 1 - Compute image descriptors (QS1)

### Calcular els histogrames de totes les imatges de la carpeta especificada a "imagesFolder"
Aquí la idea es fer el bucle que recorre totes les imatges i calcula el histograma de cadascuna d'elles, afegint el resultat a una matriu d'histogrames que després ens servirán per calcular les "Similarity measures".
Cada fila correspondrà a una imatge. 

In [ ]:
imagesFolder = "./bbdd/"
histogram_bbdd_matrix = np.empty([0, 256*3]) #Creem una matriu buida
print(histogram_bbdd_matrix.shape)
for filename in sorted(listdir(imagesFolder)):
    if(filename != '.DS_Store'):
        print("******************************************************************************")
        print("IMAGE: " + filename)
        print("******************************************************************************")

        img = cv2.imread(imagesFolder + filename)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2Lab)
        
        colorx = ('x','y','z')
        hist_img = np.empty([0,0])
        for i,col in enumerate(colorx):
            hist = cv2.calcHist([img],[i],None,[256],[0,256]) #Calculem histogrames
            hist = preprocessing.normalize(hist, norm='max') #Normalitzem histogrames
            hist_t = hist.transpose()
            print(hist_t.shape)
            if i == 0:
                hist_img = hist_t
            else:
                hist_img = np.concatenate((hist_img, hist_t), axis = 1)
            
        histogram_bbdd_matrix = np.vstack((histogram_bbdd_matrix, hist_img))   

In [ ]:
print(histogram_bbdd_matrix.shape)

### Calcular els histogrames de totes les imatges de la carpeta de query (qsd1_w1)

In [ ]:
queryFolder = "./qsd1_w1/"
histogram_query_matrix = np.empty([0, 256*3])
for filename in sorted(listdir(queryFolder)):
    if(filename != '.DS_Store' and (filename.split('.')[1] == 'jpg' or filename.split('.')[1] == 'png')):
        print("******************************************************************************")
        print("./qsd1_w1/" + filename)
        print("******************************************************************************")

        query_img = cv2.imread(queryFolder + filename)
        query_img = cv2.cvtColor(query_img,cv2.COLOR_BGR2Lab)
        color = ('b','g','r')
        hist_query = np.empty([0, 256*3])
        
        for i,col in enumerate(color):
            histr = cv2.calcHist([query_img],[i],None,[256],[0,256])
            histr = preprocessing.normalize(histr, norm='max')
            histr = histr.transpose()
            if i == 0:
                hist_query = histr
            else:
                hist_query = np.concatenate((hist_query,histr), axis = 1)
        
        histogram_query_matrix = np.vstack((histogram_query_matrix, hist_query))
    


In [ ]:
print(histogram_query_matrix.shape)

# TASK 2 - Similarity measures

### Euclidean distance

In [ ]:
from scipy.spatial import distance

dst = np.zeros((30, 279))
matrix_retrieval = np.zeros((30,10))

for query_image_index in range(0,len(histogram_query_matrix)): 
    for bbdd_image_index in range(0, len(histogram_bbdd_matrix)):
        
        #Euclidian distance 1
        dst[query_image_index,bbdd_image_index] = distance.euclidean(histogram_query_matrix[query_image_index,:], histogram_bbdd_matrix[bbdd_image_index,:])
        
        #Euclidian distance 2
        dst[query_image_index,bbdd_image_index] = np.linalg.norm(histogram_query_matrix[query_image_index,:] - histogram_bbdd_matrix[bbdd_image_index,:], ord=1)
        
        #Bray Curtis Distance
        dst[query_image_index,bbdd_image_index] = distance.braycurtis(histogram_query_matrix[query_image_index,:], histogram_bbdd_matrix[bbdd_image_index,:])
    
    matrix_retrieval[query_image_index,:] = np.argsort(dst[query_image_index,:])[:10]

print(matrix_retrieval)

# Task 4

In [ ]:
import ml_metrics as metrics
import pickle

with open('./qsd1_w1/gt_corresps.pkl', 'rb') as fd:
        ll = pickle.load(fd)
        print(ll)

#Convertir idx(numpy) a list of lists
matrix_retrieval_lst = matrix_retrieval.tolist()

#print(matrix_retrieval_lst)

ll_prp = np.zeros((len(ll),1))

for i in range(0,len(ll)):
    ll_prp[i] = ll[i][0][1]

ll_prp_lst = ll_prp.tolist()

mapak = metrics.average_precision.mapk(ll_prp_lst, matrix_retrieval_lst, k=10)        

print(mapak)

# Task 5

In [ ]:
queryFolder = "./qsd2_w1/"
for filename in sorted(listdir(queryFolder)):
    if(filename != '.DS_Store' and (filename.split('.')[1] == 'jpg')):
        print("******************************************************************************")
        print("./qsd2_w1/" + filename)
        print("******************************************************************************")

        query_img2 = cv2.imread(queryFolder + filename)
        gray = cv2.cvtColor(query_img2, cv2.COLOR_BGR2GRAY)
        
        sobel = filters.sobel(gray)
        
        plt.rcParams['image.interpolation'] = 'nearest'
        plt.rcParams['image.cmap'] = 'gray'
        plt.rcParams['figure.dpi'] = 200
        
        blurred = filters.gaussian(sobel, sigma=2.0)
        
        med = filters.median(blurred)
        thresh, blackAndWhiteImage = cv2.threshold(med, 5, 255, cv2.THRESH_BINARY)
        
        maximum = np.max(blurred)
        minimum = np.min(blurred)

        kernel = np.ones((5, 5), np.uint8)
        erosion1 = cv2.erode(blackAndWhiteImage, kernel, iterations=1)
        dilation1 = cv2.dilate(erosion1, kernel, iterations=1)

        cv2.imwrite(query_img2, blackAndWhiteImage)
    

In [ ]:
query_img2 = cv2.imread('./qsd2_w1/00007.jpg')
gray = cv2.cvtColor(query_img2, cv2.COLOR_BGR2GRAY)
        
sobel = filters.sobel(gray)
        
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['figure.dpi'] = 200

blurred = filters.gaussian(sobel, sigma=2.0)
        
med = filters.median(blurred)
thresh, blackAndWhiteImage = cv2.threshold(med, 5, 255, cv2.THRESH_BINARY)
        
maximum = np.max(blurred)
minimum = np.min(blurred)

kernel = np.ones((5, 5), np.uint8)
erosion1 = cv2.erode(blackAndWhiteImage, kernel, iterations=1)
dilation1 = cv2.dilate(erosion1, kernel, iterations=1)

plt.imshow(dilation1)
#cv2.imwrite(query_img2, blackAndWhiteImage)
    

In [ ]:
query_img2 = cv2.imread('./qsd2_w1/00000.jpg')
gray = cv2.cvtColor(query_img2, cv2.COLOR_BGR2GRAY)

ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

# Marker labelling
ret, markers = cv2.connectedComponents(sure_bg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv2.watershed(query_img2,markers)
query_img2[markers == -1] = [255,0,0]

plt.imshow(markers)


In [ ]:
print(query_img2.shape)

In [ ]:
print(len(query_img2))

In [ ]:
query_img2 = cv2.imread('./qsd2_w1/00000.jpg')
gray = cv2.cvtColor(query_img2, cv2.COLOR_BGR2GRAY)

shifted = cv2.pyrMeanShiftFiltering(query_img2, 250, 50)

gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

plt.imshow(thresh)


